Import Libraries

In [1]:
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
from scipy import interpolate

# Import pandas
import pandas as pd

# Import numpy
import numpy as np
from numpy import * 
from numpy.linalg import multi_dot
import numpy_financial as npf

# Plot settings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = 16, 8

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# IBT or Pension or Retirement
PLAN = 'Retirement'

UPS_Contr_Pctg = 0

In [2]:
df_CF = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "PBO Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.5,1.204791e+09,2.626341e+08,4.239987e+08
2022-12-31,1.5,1.261684e+09,2.935988e+08,4.570359e+08
2023-12-31,2.5,1.322496e+09,3.253961e+08,4.883641e+08
2024-12-31,3.5,1.382473e+09,3.576365e+08,5.168109e+08
2025-12-31,4.5,1.441835e+09,3.898680e+08,5.408954e+08
...,...,...,...,...
2096-12-31,75.5,1.769334e+05,3.948190e+06,1.620103e+06
2097-12-31,76.5,1.503230e+05,2.959047e+06,1.183430e+06
2098-12-31,77.5,1.300395e+05,2.180928e+06,8.499443e+05


In [3]:
df_CF = df_CF.divide(12)
df_CF

,Time,Retirement,Pension,IBT
Year,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-12-31,0.125000,1.051403e+08,2.446657e+07,3.808632e+07
2023-12-31,0.208333,1.102080e+08,2.711634e+07,4.069701e+07
2024-12-31,0.291667,1.152061e+08,2.980304e+07,4.306757e+07
2025-12-31,0.375000,1.201529e+08,3.248900e+07,4.507462e+07
...,...,...,...,...
2096-12-31,6.291667,1.474445e+04,3.290158e+05,1.350085e+05
2097-12-31,6.375000,1.252692e+04,2.465872e+05,9.861913e+04
2098-12-31,6.458333,1.083662e+04,1.817440e+05,7.082869e+04


In [4]:
df=df_CF.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_CF = df
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-02-28,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-03-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-04-30,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [5]:
df_CF_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "CF", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,19389650.5,31075638.75
2021-11-30,0.166667,9.387313e+07,19389650.5,31075638.75
2021-12-31,0.250000,9.387313e+07,19389650.5,31075638.75


In [6]:
df_CF  = df_CF_add.append(df_CF)
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.041667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,6.541667,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,6.541667,9.606686e+03,1.315861e+05,4.997286e+04


In [7]:
from itertools import count, takewhile
def frange(start, stop, step):
        return takewhile(lambda x: x< stop, count(start, step))
t = list(frange((1/12), 79.41666666666, (1/12)))

In [8]:
df_t=pd.DataFrame(t)
df_t

,0
0,0.083333
1,0.166667
2,0.250000
3,0.333333
4,0.416667
...,...
947,79.000000
948,79.083333
949,79.166667
950,79.250000


In [9]:
df_CF['Time'] = df_t.values
df_CF

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [10]:
#IBT = df_CF[PLAN]
#RP = df_CF[PLAN]
CF = np.array(df_CF[PLAN])
Time_CF = df_CF.index
DF = np.array(df_CF['Time'])
CF

array([9.38731288e+07, 9.38731288e+07, 9.38731288e+07, 1.00399288e+08,
       1.00399288e+08, 1.00399288e+08, 1.00399288e+08, 1.00399288e+08,
       1.00399288e+08, 1.00399288e+08, 1.00399288e+08, 1.00399288e+08,
       1.00399288e+08, 1.00399288e+08, 1.00399288e+08, 1.05140342e+08,
       1.05140342e+08, 1.05140342e+08, 1.05140342e+08, 1.05140342e+08,
       1.05140342e+08, 1.05140342e+08, 1.05140342e+08, 1.05140342e+08,
       1.05140342e+08, 1.05140342e+08, 1.05140342e+08, 1.10208005e+08,
       1.10208005e+08, 1.10208005e+08, 1.10208005e+08, 1.10208005e+08,
       1.10208005e+08, 1.10208005e+08, 1.10208005e+08, 1.10208005e+08,
       1.10208005e+08, 1.10208005e+08, 1.10208005e+08, 1.15206053e+08,
       1.15206053e+08, 1.15206053e+08, 1.15206053e+08, 1.15206053e+08,
       1.15206053e+08, 1.15206053e+08, 1.15206053e+08, 1.15206053e+08,
       1.15206053e+08, 1.15206053e+08, 1.15206053e+08, 1.20152905e+08,
       1.20152905e+08, 1.20152905e+08, 1.20152905e+08, 1.20152905e+08,
      

In [11]:
from PyCurve.curve import Curve

In [12]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 145)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,2010-09-30 00:00:00,2010-08-31 00:00:00,2010-07-31 00:00:00,2010-06-30 00:00:00,2010-05-31 00:00:00,2010-04-30 00:00:00,2010-03-31 00:00:00,2010-02-28 00:00:00,2010-01-31 00:00:00,2009-12-31 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,0.896931,0.980827,1.039656,1.336495,1.356106,0.796529,0.824371,0.959171,0.854822,0.997499
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,1.012479,1.093406,1.156291,1.494602,1.548488,1.069795,1.179135,1.183362,1.099006,1.386799
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,1.065754,1.180900,1.244818,1.590332,1.686592,1.323298,1.459969,1.412837,1.358586,1.664896
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,1.132115,1.250318,1.346470,1.693166,1.853045,1.533786,1.684716,1.612329,1.567095,1.892898
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,1.289564,1.400126,1.517540,1.883170,2.099087,1.825434,1.982464,1.898602,1.860517,2.202034
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,1.480375,1.593382,1.723042,2.141054,2.396014,2.163991,2.328185,2.229761,2.204905,2.555180
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,1.695568,1.812045,1.960053,2.417479,2.679186,2.485899,2.676519,2.544241,2.541772,2.898791
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,1.924112,2.035995,2.214525,2.671984,2.922165,2.770801,2.999055,2.823711,2.849782,3.213513
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,2.149881,2.249067,2.462229,2.882171,3.138009,3.038841,3.281442,3.099499,3.139972,3.515488
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,2.374888,2.459324,2.706179,3.066176,3.344929,3.302149,3.534706,3.380740,3.421418,3.802621


In [13]:
raw_liab_ibt_dict={}
tr = Raw_FTSE_YC['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict[col])
#curve = Curve(t,y)
#print("curve.get_rate:{},curve.get_time:{}".format(curve.get_rate,curve.get_time))
#print(curve.get_rate)
#print(curve.get_time)

952

In [14]:
int_df = pd.DataFrame(raw_liab_ibt_dict)
int_df = int_df.drop(['Date'], axis=1)
int_df = int_df.iloc[:, ::-1]
int_df

,2009-12-31,2010-01-31,2010-02-28,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
1,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
2,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
3,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
4,0.997499,0.854822,0.959171,0.824371,0.796529,1.356106,1.336495,1.039656,0.980827,0.896931,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
948,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
949,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
950,6.204541,6.310049,6.416999,6.458114,6.171532,6.254318,5.862119,5.847708,5.527934,5.623430,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599


In [15]:
liab_plan_dict={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df[col][j]/100)**DF[j]))
    liab_plan_dict[col] = temp_pv
len(liab_plan_dict)

142

In [16]:
pv_df = pd.DataFrame(liab_plan_dict, index = ['Present Values']).transpose()
np.negative(pv_df['Present Values'])
pv_df

,Present Values
2009-12-31,2.282568e+10
2010-01-31,2.310559e+10
2010-02-28,2.298846e+10
2010-03-31,2.280439e+10
2010-04-30,2.351785e+10
...,...
2021-05-31,3.365854e+10
2021-06-30,3.479317e+10
2021-07-31,3.552982e+10
2021-08-31,3.524508e+10


In [17]:
Plan_Return_YC = np.zeros(len(liab_plan_dict)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC[i] = ((pv_df['Present Values'][i+1])/pv_df['Present Values'][i])-1
Plan_Return_YC

array([ 0.01226323, -0.0050695 , -0.00800702,  0.03128594, -0.00363323,
        0.04355224,  0.01151837,  0.05102676, -0.01156774, -0.03017955,
       -0.0074885 , -0.01965443, -0.02375441,  0.01383598, -0.01180002,
        0.02083923,  0.01686683, -0.02404912,  0.04110074,  0.02058869,
        0.05289754,  0.00061807, -0.00022053,  0.03887863,  0.00216995,
        0.00098562, -0.03010959,  0.02126239,  0.02670519,  0.02544698,
        0.05379146, -0.00992935, -0.01394655,  0.01640673, -0.01007208,
       -0.01786008, -0.03610423,  0.00718574, -0.00711066,  0.03355196,
       -0.05796417, -0.0416115 , -0.0016925 , -0.00796243, -0.00266235,
        0.0155544 , -0.0150099 , -0.00620677,  0.03449413,  0.01190134,
        0.00336946,  0.01394905,  0.01756789, -0.00329922, -0.00068462,
        0.02995825, -0.02850853,  0.01669129,  0.01501217,  0.01489509,
        0.06441217, -0.03559421,  0.00079539, -0.02432492, -0.02386286,
       -0.0381149 ,  0.01724172, -0.01089622,  0.01070085,  0.00

In [18]:
Plan_Returns_YC = pd.DataFrame(Plan_Return_YC)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC.to_excel(writer, sheet_name='returns')
pv_df.to_excel(writer, sheet_name='pv')
writer.save()

In [19]:
df_SC = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "Service Cost Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC

,Retirement,Pension,IBT
Year,,,
2021-12-31,1511573.39,1.515507e+05,8.562248e+05
2022-12-31,4341798.12,6.378839e+05,3.265973e+06
2023-12-31,6325185.11,1.364457e+06,6.377871e+06
2024-12-31,8362114.74,2.184367e+06,9.520245e+06
2025-12-31,10514285.02,3.091714e+06,1.271774e+07
...,...,...,...
2096-12-31,2534.24,7.448368e+05,8.144005e+05
2097-12-31,1502.48,5.765053e+05,6.166164e+05
2098-12-31,873.21,4.384455e+05,4.589457e+05


In [20]:
df_SC = df_SC.divide(12)
df_SC

,Retirement,Pension,IBT
Year,,,
2021-12-31,125964.449167,12629.223266,7.135207e+04
2022-12-31,361816.510000,53156.995293,2.721644e+05
2023-12-31,527098.759167,113704.742421,5.314892e+05
2024-12-31,696842.895000,182030.602914,7.933538e+05
2025-12-31,876190.418333,257642.865694,1.059812e+06
...,...,...,...
2096-12-31,211.186667,62069.734330,6.786670e+04
2097-12-31,125.206667,48042.110266,5.138470e+04
2098-12-31,72.767500,36537.126910,3.824548e+04


In [21]:
df=df_SC.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_SC = df
df_SC

,Retirement,Pension,IBT
date,,,
2021-12-31,125964.449167,12629.223266,71352.069903
2022-01-31,125964.449167,12629.223266,71352.069903
2022-02-28,125964.449167,12629.223266,71352.069903
2022-03-31,125964.449167,12629.223266,71352.069903
2022-04-30,125964.449167,12629.223266,71352.069903
...,...,...,...
2100-08-31,41.463333,27272.299072,27955.270099
2100-09-30,41.463333,27272.299072,27955.270099
2100-10-31,41.463333,27272.299072,27955.270099


In [22]:
df_SC_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "SC", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC_add

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333


In [23]:
df_SC  = df_SC_add.append(df_SC)
df_SC

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,184557.083333,4629.916667,70374.833333
2021-11-30,0.166667,184557.083333,4629.916667,70374.833333
2021-12-31,0.250000,184557.083333,4629.916667,70374.833333
2021-12-31,NaN,125964.449167,12629.223266,71352.069903
2022-01-31,NaN,125964.449167,12629.223266,71352.069903
...,...,...,...,...
2100-08-31,NaN,41.463333,27272.299072,27955.270099
2100-09-30,NaN,41.463333,27272.299072,27955.270099
2100-10-31,NaN,41.463333,27272.299072,27955.270099


In [24]:
col_SC = df_SC.columns
col_CF = df_CF.columns
df_CF_Tot = UPS_Contr_Pctg*df_SC[col_SC] + df_CF[col_CF]
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,NaN,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,NaN,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,NaN,9.606686e+03,1.315861e+05,4.997286e+04


In [25]:
df_CF_Tot['Time'] = df_t.values
df_CF_Tot

,Time,Retirement,Pension,IBT
date,,,,
2021-10-31,0.083333,9.387313e+07,1.938965e+07,3.107564e+07
2021-11-30,0.166667,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.250000,9.387313e+07,1.938965e+07,3.107564e+07
2021-12-31,0.333333,1.003993e+08,2.188617e+07,3.533322e+07
2022-01-31,0.416667,1.003993e+08,2.188617e+07,3.533322e+07
...,...,...,...,...
2100-08-31,79.000000,9.606686e+03,1.315861e+05,4.997286e+04
2100-09-30,79.083333,9.606686e+03,1.315861e+05,4.997286e+04
2100-10-31,79.166667,9.606686e+03,1.315861e+05,4.997286e+04


In [26]:
CF_Tot = df_CF_Tot[PLAN]
Time_CF_Tot = df_CF_Tot.index
DF_Tot=df_CF_Tot['Time']

In [27]:
liab_plan_dict_tot={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot[j]/((1+int_df[col][j]/100)**DF_Tot[j]))
    liab_plan_dict_tot[col] = temp_pv
liab_plan_dict_tot


{datetime.datetime(2009, 12, 31, 0, 0): 22825678131.721,
 datetime.datetime(2010, 1, 31, 0, 0): 23105594563.939457,
 datetime.datetime(2010, 2, 28, 0, 0): 22988460684.92035,
 datetime.datetime(2010, 3, 31, 0, 0): 22804391719.988926,
 datetime.datetime(2010, 4, 30, 0, 0): 23517848657.368317,
 datetime.datetime(2010, 5, 31, 0, 0): 23432402836.089603,
 datetime.datetime(2010, 6, 30, 0, 0): 24452936507.713783,
 datetime.datetime(2010, 7, 31, 0, 0): 24734594583.82833,
 datetime.datetime(2010, 8, 31, 0, 0): 25996720877.541187,
 datetime.datetime(2010, 9, 30, 0, 0): 25695997697.618694,
 datetime.datetime(2010, 10, 31, 0, 0): 24920504089.48789,
 datetime.datetime(2010, 11, 30, 0, 0): 24733886899.815865,
 datetime.datetime(2010, 12, 31, 0, 0): 24247756392.340206,
 datetime.datetime(2011, 1, 31, 0, 0): 23671765282.83585,
 datetime.datetime(2011, 2, 28, 0, 0): 23999287413.002308,
 datetime.datetime(2011, 3, 31, 0, 0): 23716095268.23378,
 datetime.datetime(2011, 4, 30, 0, 0): 24210320395.806942,
 

In [28]:
pv_df_tot = pd.DataFrame(liab_plan_dict_tot, index = ['Present Values']).transpose()

pv_df_tot

,Present Values
2009-12-31,2.282568e+10
2010-01-31,2.310559e+10
2010-02-28,2.298846e+10
2010-03-31,2.280439e+10
2010-04-30,2.351785e+10
...,...
2021-05-31,3.365854e+10
2021-06-30,3.479317e+10
2021-07-31,3.552982e+10
2021-08-31,3.524508e+10


In [29]:
Plan_Return_YC_Tot = np.zeros(len(liab_plan_dict_tot)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Tot[i] = ((pv_df_tot['Present Values'][i+1])/pv_df_tot['Present Values'][i])-1
Plan_Returns_YC_Tot = pd.DataFrame(Plan_Return_YC_Tot)
Plan_Returns_YC_Tot

,0
0,0.012263
1,-0.005070
2,-0.008007
3,0.031286
4,-0.003633
...,...
136,0.002867
137,0.033710
138,0.021172
139,-0.008014


In [30]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability & SC Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Tot.to_excel(writer, sheet_name='returns')
pv_df_tot.to_excel(writer, sheet_name='pv')
writer.save()

Calculate IRR's

In [31]:
Plan_DR = pd.read_excel("UPS Pension - Historical Liability Data - 9.30.21.xlsx",sheet_name = PLAN ,skiprows = [0,1,2,3],usecols=[1,2], na_values=[""],index_col=0)
#dataset.set_index('Year')
Plan_DR


,IRR
Date,
2009-12-31,0.059373
2010-01-31,0.058109
2010-02-28,0.058852
2010-03-31,0.059173
2010-04-30,0.056593
...,...
2021-05-31,0.029939
2021-06-30,0.027828
2021-07-31,0.026312


In [32]:
#col_DR = Plan_DR.columns
#df_DR_M =  (1+Plan_DR)**(1/12)-1
#df_DR_M = Plan_DR
IRR = Plan_DR['IRR']
IRR

Date
2009-12-31    0.059373
2010-01-31    0.058109
2010-02-28    0.058852
2010-03-31    0.059173
2010-04-30    0.056593
                ...   
2021-05-31    0.029939
2021-06-30    0.027828
2021-07-31    0.026312
2021-08-31    0.026872
2021-09-30    0.030700
Name: IRR, Length: 142, dtype: float64

In [33]:
#liab_plan_dict_dr={}
#for col in Plan_DR.columns:
    #print('col: {}'.format(col))
#temp_pv=0
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
    #for j in range (0,len(Time_CF)):
        temp_pv += (CF[i]/((1+Plan_DR['IRR'][j]**DF[i])))
    #liab_plan_dict_dr[col] = temp_pv
    PV_DR = temp_pv
PV_DR

7816977877406.272

In [34]:
temp_pv = (CF[0]/((1+Plan_DR['IRR'][-1])**DF[0]))
temp_pv

93636880.54212457

In [35]:
Plan_PV_DR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
        Plan_PV_DR[j] += (CF[i]/((1+Plan_DR['IRR'][j])**DF[i]))
Plan_PV_DR

array([2.24216584e+10, 2.27609135e+10, 2.25605228e+10, 2.24747238e+10,
       2.31786269e+10, 2.28507176e+10, 2.38149092e+10, 2.42181008e+10,
       2.53041819e+10, 2.50953247e+10, 2.44095636e+10, 2.41685431e+10,
       2.38791059e+10, 2.33027422e+10, 2.36978131e+10, 2.34146758e+10,
       2.39357161e+10, 2.42887458e+10, 2.37176961e+10, 2.44784580e+10,
       2.48193258e+10, 2.56988478e+10, 2.58654606e+10, 2.59356699e+10,
       2.69276068e+10, 2.70299810e+10, 2.71405598e+10, 2.64663430e+10,
       2.69983714e+10, 2.76290326e+10, 2.84988057e+10, 2.99658603e+10,
       2.97911956e+10, 2.94718741e+10, 2.99853172e+10, 2.96996676e+10,
       2.92086565e+10, 2.81823098e+10, 2.84131325e+10, 2.81828955e+10,
       2.91260640e+10, 2.74796295e+10, 2.63050306e+10, 2.63288780e+10,
       2.61790387e+10, 2.61175932e+10, 2.65730338e+10, 2.61905518e+10,
       2.60120336e+10, 2.69563349e+10, 2.72873182e+10, 2.73881241e+10,
       2.77602468e+10, 2.82643062e+10, 2.82122534e+10, 2.81714618e+10,
      

In [36]:
pv_df_dr = pd.DataFrame(Plan_PV_DR, columns=['Present Values'])
#np.negative(pv_df_dr['Present Values'])
test = np.append(np.negative(pv_df_dr['Present Values'][0]),CF)
#pv_df_dr['Present Values']
test

array([-2.24216584e+10,  9.38731288e+07,  9.38731288e+07,  9.38731288e+07,
        1.00399288e+08,  1.00399288e+08,  1.00399288e+08,  1.00399288e+08,
        1.00399288e+08,  1.00399288e+08,  1.00399288e+08,  1.00399288e+08,
        1.00399288e+08,  1.00399288e+08,  1.00399288e+08,  1.00399288e+08,
        1.05140342e+08,  1.05140342e+08,  1.05140342e+08,  1.05140342e+08,
        1.05140342e+08,  1.05140342e+08,  1.05140342e+08,  1.05140342e+08,
        1.05140342e+08,  1.05140342e+08,  1.05140342e+08,  1.05140342e+08,
        1.10208005e+08,  1.10208005e+08,  1.10208005e+08,  1.10208005e+08,
        1.10208005e+08,  1.10208005e+08,  1.10208005e+08,  1.10208005e+08,
        1.10208005e+08,  1.10208005e+08,  1.10208005e+08,  1.10208005e+08,
        1.15206053e+08,  1.15206053e+08,  1.15206053e+08,  1.15206053e+08,
        1.15206053e+08,  1.15206053e+08,  1.15206053e+08,  1.15206053e+08,
        1.15206053e+08,  1.15206053e+08,  1.15206053e+08,  1.15206053e+08,
        1.20152905e+08,  

In [37]:
#pv_df_dr = pd.DataFrame(Plan_PV_DR)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability PV Using DR.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
pv_df_dr.to_excel(writer, sheet_name='Present Values')
writer.save()

In [38]:
import numpy as np
from scipy.optimize import fsolve

def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

def irr(cfs, yrs, x0, **kwargs):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs), **kwargs))

In [39]:
IRR = np.zeros(len(pv_df))
for j in range (len(pv_df)):
    cash_flow = np.append(np.negative(pv_df['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.05787064, 0.05685503, 0.05727755, 0.05794871, 0.05539539,
       0.05569447, 0.05223423, 0.05132026, 0.04742259, 0.04832323,
       0.05072617, 0.05132253, 0.05291086, 0.0548611 , 0.0537428 ,
       0.05470827, 0.05303532, 0.05169437, 0.05365062, 0.0504297 ,
       0.04883032, 0.04487246, 0.04482576, 0.04484243, 0.04199213,
       0.04183207, 0.04175939, 0.04403462, 0.04246443, 0.04052482,
       0.03870302, 0.03498806, 0.03568709, 0.03667765, 0.03553062,
       0.03624289, 0.03752111, 0.04017067, 0.03965033, 0.04016892,
       0.03778851, 0.0421293 , 0.04531307, 0.04544164, 0.04605017,
       0.04625375, 0.04507959, 0.04622999, 0.04670664, 0.04413189,
       0.04324584, 0.04299505, 0.0419675 , 0.0406875 , 0.04092939,
       0.04097957, 0.03883452, 0.04093594, 0.0397289 , 0.03865227,
       0.03759308, 0.03321824, 0.03573947, 0.03568361, 0.03742563,
       0.03915819, 0.04199727, 0.04074041, 0.04154448, 0.04076324,
       0.0405984 , 0.04072479, 0.04104437, 0.04041136, 0.03904

In [40]:
IRR = np.zeros(len(pv_df_tot))
for j in range (len(pv_df_tot)):
    cash_flow = np.append(np.negative(pv_df_tot['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.05787064, 0.05685503, 0.05727755, 0.05794871, 0.05539539,
       0.05569447, 0.05223423, 0.05132026, 0.04742259, 0.04832323,
       0.05072617, 0.05132253, 0.05291086, 0.0548611 , 0.0537428 ,
       0.05470827, 0.05303532, 0.05169437, 0.05365062, 0.0504297 ,
       0.04883032, 0.04487246, 0.04482576, 0.04484243, 0.04199213,
       0.04183207, 0.04175939, 0.04403462, 0.04246443, 0.04052482,
       0.03870302, 0.03498806, 0.03568709, 0.03667765, 0.03553062,
       0.03624289, 0.03752111, 0.04017067, 0.03965033, 0.04016892,
       0.03778851, 0.0421293 , 0.04531307, 0.04544164, 0.04605017,
       0.04625375, 0.04507959, 0.04622999, 0.04670664, 0.04413189,
       0.04324584, 0.04299505, 0.0419675 , 0.0406875 , 0.04092939,
       0.04097957, 0.03883452, 0.04093594, 0.0397289 , 0.03865227,
       0.03759308, 0.03321824, 0.03573947, 0.03568361, 0.03742563,
       0.03915819, 0.04199727, 0.04074041, 0.04154448, 0.04076324,
       0.0405984 , 0.04072479, 0.04104437, 0.04041136, 0.03904

In [41]:
IRR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    cash_flow = np.append(np.negative(pv_df_dr['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

array([0.05937312, 0.05810854, 0.05885172, 0.05917325, 0.05659336,
       0.05777896, 0.05436927, 0.05300942, 0.04952246, 0.05017402,
       0.05237649, 0.05317458, 0.05415025, 0.05615174, 0.05477124,
       0.05575675, 0.05395791, 0.05277495, 0.05470276, 0.05215071,
       0.0510486 , 0.04831461, 0.04781363, 0.04760408, 0.04473753,
       0.0444513 , 0.04414406, 0.0460491 , 0.04453949, 0.0428104 ,
       0.04052615, 0.03691304, 0.03732847, 0.03809795, 0.036867  ,
       0.0375477 , 0.03874217, 0.04134439, 0.04074621, 0.04134287,
       0.0389462 , 0.0432143 , 0.04651645, 0.04644707, 0.04688468,
       0.0470653 , 0.0457425 , 0.04685091, 0.04737718, 0.04465703,
       0.04373938, 0.04346341, 0.04245853, 0.04113101, 0.04126636,
       0.0413727 , 0.03925939, 0.04136065, 0.04036554, 0.03921769,
       0.03830584, 0.03412832, 0.03661942, 0.03647775, 0.03818184,
       0.03965781, 0.0424847 , 0.04131521, 0.0422186 , 0.04186758,
       0.04150519, 0.04153418, 0.04202704, 0.04182906, 0.04053

In [42]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(14, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


,2009-11-30,2009-10-31,2009-09-30,2009-08-31,2009-07-31,2009-06-30,2009-05-31,2009-04-30,2009-03-31,2009-02-28,...,1996-06-30,1996-05-31,1996-04-30,1996-03-31,1996-02-29,1996-01-31,1995-12-31,1995-11-30,1995-10-31,1995-09-30
0,1.00,1.04,1.20,1.37,1.65,2.08,2.56,3.12,4.42,4.39,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.15,1.32,1.49,1.63,1.98,2.36,2.78,3.41,4.63,4.62,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.33,1.58,1.76,1.91,2.27,2.60,2.95,3.57,4.65,4.64,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,1.51,1.79,1.97,2.12,2.49,2.81,3.14,3.76,4.74,4.80,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,1.78,2.09,2.26,2.40,2.77,3.11,3.43,4.05,4.95,5.04,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,2.08,2.43,2.57,2.73,3.08,3.46,3.75,4.35,5.20,5.30,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,2.39,2.77,2.87,3.03,3.38,3.79,4.08,4.60,5.38,5.49,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,2.70,3.06,3.16,3.31,3.66,4.10,4.41,4.81,5.50,5.62,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,3.02,3.34,3.46,3.59,3.93,4.39,4.75,5.03,5.63,5.76,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,3.33,3.61,3.75,3.87,4.20,4.66,5.08,5.27,5.81,5.95,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [43]:
Raw_FTSE_YC_Tot = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot

,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,1996-06-30 00:00:00,1996-05-31 00:00:00,1996-04-30 00:00:00,1996-03-31 00:00:00,1996-02-29 00:00:00,1996-01-31 00:00:00,1995-12-31 00:00:00,1995-11-30 00:00:00,1995-10-31 00:00:00,1995-09-30 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [44]:
raw_liab_ibt_dict_tot={}
tr = Raw_FTSE_YC_Tot['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot[col])

952

In [45]:
int_df_tot = pd.DataFrame(raw_liab_ibt_dict_tot)
int_df_tot = int_df_tot.drop(['Date'], axis=1)
int_df_tot = int_df_tot.iloc[:, ::-1]
int_df_tot

,1995-09-30,1995-10-31,1995-11-30,1995-12-31,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,...,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
0,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
1,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
2,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
3,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
4,5.98,5.96,5.87,5.63,5.41,5.52,5.71,5.75,5.78,5.81,...,0.288271,0.262039,0.218973,0.241936,0.215197,0.185391,0.229425,0.225192,0.219006,0.228012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
948,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
949,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599
950,7.34,7.25,7.09,6.79,6.98,7.65,7.80,8.11,8.06,7.87,...,2.717421,2.946016,3.185627,3.382834,3.254473,3.249005,2.984096,2.859711,2.903617,3.020599


In [46]:
liab_plan_dict_totdf={}
for col in int_df_tot.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot[col][j]/100)**DF[j]))
    liab_plan_dict_totdf[col] = temp_pv
liab_plan_dict_totdf

{Timestamp('1995-09-30 00:00:00'): 19347976249.798626,
 Timestamp('1995-10-31 00:00:00'): 19663591272.85913,
 Timestamp('1995-11-30 00:00:00'): 20119961742.474697,
 Timestamp('1995-12-31 00:00:00'): 20598895443.988457,
 Timestamp('1996-01-31 00:00:00'): 20478760496.13397,
 Timestamp('1996-02-29 00:00:00'): 19358043733.79895,
 Timestamp('1996-03-31 00:00:00'): 18890359333.32342,
 Timestamp('1996-04-30 00:00:00'): 18347095753.541405,
 Timestamp('1996-05-31 00:00:00'): 18241893716.370724,
 Timestamp('1996-06-30 00:00:00'): 18476392069.191357,
 Timestamp('1996-07-31 00:00:00'): 18354733995.422653,
 Timestamp('1996-08-31 00:00:00'): 18047494591.211964,
 Timestamp('1996-09-30 00:00:00'): 18485155154.390656,
 Timestamp('1996-10-31 00:00:00'): 19167693964.99834,
 Timestamp('1996-11-30 00:00:00'): 19659955709.945225,
 Timestamp('1996-12-31 00:00:00'): 19063385146.146706,
 Timestamp('1997-01-31 00:00:00'): 18832379726.430473,
 Timestamp('1997-02-28 00:00:00'): 18776800051.33405,
 Timestamp('1997

In [47]:
pv_df_totdf= pd.DataFrame(liab_plan_dict_totdf, index = ['Present Values']).transpose()
np.negative(pv_df_totdf['Present Values'])
pv_df_totdf.tail

<bound method NDFrame.tail of             Present Values
1995-09-30    1.934798e+10
1995-10-31    1.966359e+10
1995-11-30    2.011996e+10
1995-12-31    2.059890e+10
1996-01-31    2.047876e+10
...                    ...
2021-05-31    3.365854e+10
2021-06-30    3.479317e+10
2021-07-31    3.552982e+10
2021-08-31    3.524508e+10
2021-09-30    3.456585e+10

[313 rows x 1 columns]>

In [48]:
Plan_Return_YC_Totdf = np.zeros(len(liab_plan_dict_totdf)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf[i] = ((pv_df_totdf['Present Values'][i+1])/pv_df_totdf['Present Values'][i])-1
Plan_Returns_YC_Totdf = pd.DataFrame(Plan_Return_YC_Totdf)
Plan_Returns_YC_Totdf

,0
0,0.016313
1,0.023209
2,0.023804
3,-0.005832
4,-0.054726
...,...
307,0.002867
308,0.033710
309,0.021172
310,-0.008014


In [49]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf.to_excel(writer, sheet_name='returns')
pv_df_totdf.to_excel(writer, sheet_name='pv')
writer.save()

Test Liability Return Data Pre-2010

In [50]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 130)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,2011-10-31 00:00:00,2011-09-30 00:00:00,2011-08-31 00:00:00,2011-07-31 00:00:00,2011-06-30 00:00:00,2011-05-31 00:00:00,2011-04-30 00:00:00,2011-03-31 00:00:00,2011-02-28 00:00:00,2011-01-31 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,0.978098,1.200821,1.016695,0.834075,0.740336,0.691775,0.651798,0.684030,0.800783,0.855293
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,1.058504,1.289367,1.102144,0.904379,0.884920,0.798170,0.811379,0.933030,0.939382,0.966797
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,1.104752,1.336674,1.144379,0.926479,1.013526,0.896584,0.959028,1.175297,1.109245,1.082385
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,1.160063,1.394062,1.164494,0.970688,1.138191,1.032233,1.135948,1.380596,1.287303,1.229226
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,1.357872,1.592194,1.335450,1.155110,1.392998,1.285085,1.422351,1.695042,1.587769,1.486277
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,1.577038,1.808680,1.549877,1.382992,1.690475,1.562437,1.735382,2.049367,1.934109,1.780979
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,1.822674,2.052670,1.797824,1.664563,2.010606,1.865400,2.058998,2.393178,2.266879,2.097326
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,2.094877,2.323251,2.071329,1.985906,2.341357,2.188019,2.384139,2.711216,2.571832,2.426308
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,2.389703,2.605332,2.371506,2.317872,2.684802,2.517224,2.709798,3.024671,2.879268,2.760887
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,2.694074,2.882730,2.687323,2.642271,3.030853,2.837831,3.028868,3.337587,3.193259,3.091951


In [51]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(1, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


,2010-12-31,2010-11-30,2010-10-31,2010-09-30,2010-08-31,2010-07-31,2010-06-30,2010-05-31,2010-04-30,2010-03-31,...,1996-06-30,1996-05-31,1996-04-30,1996-03-31,1996-02-29,1996-01-31,1995-12-31,1995-11-30,1995-10-31,1995-09-30
0,0.88,0.90,0.83,0.90,0.98,1.04,1.34,1.32,0.80,0.83,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.04,0.99,0.90,1.01,1.09,1.16,1.50,1.53,1.07,1.18,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.13,1.06,0.94,1.06,1.18,1.25,1.59,1.70,1.33,1.46,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,1.27,1.13,0.97,1.13,1.25,1.35,1.70,1.86,1.54,1.69,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,1.54,1.32,1.12,1.29,1.40,1.52,1.89,2.10,1.83,1.99,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,1.87,1.56,1.31,1.48,1.60,1.73,2.15,2.40,2.17,2.34,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,2.22,1.83,1.54,1.70,1.82,1.97,2.43,2.68,2.50,2.69,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,2.56,2.11,1.81,1.93,2.05,2.23,2.69,2.93,2.79,3.02,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,2.90,2.39,2.06,2.16,2.26,2.48,2.90,3.15,3.06,3.31,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,3.22,2.67,2.31,2.39,2.48,2.73,3.09,3.36,3.33,3.57,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [52]:
Raw_FTSE_YC_Tot_1 = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot_1

,Date,2021-09-30 00:00:00,2021-08-31 00:00:00,2021-07-31 00:00:00,2021-06-30 00:00:00,2021-05-31 00:00:00,2021-04-30 00:00:00,2021-03-31 00:00:00,2021-02-28 00:00:00,2021-01-31 00:00:00,...,1996-06-30 00:00:00,1996-05-31 00:00:00,1996-04-30 00:00:00,1996-03-31 00:00:00,1996-02-29 00:00:00,1996-01-31 00:00:00,1995-12-31 00:00:00,1995-11-30 00:00:00,1995-10-31 00:00:00,1995-09-30 00:00:00
0,0.5,0.228012,0.219006,0.225192,0.229425,0.185391,0.215197,0.241936,0.218973,0.262039,...,5.81,5.78,5.75,5.71,5.52,5.41,5.63,5.87,5.96,5.98
1,1.0,0.269363,0.253146,0.250422,0.273281,0.234543,0.260351,0.292303,0.274942,0.286072,...,6.17,6.21,6.08,5.90,5.68,5.33,5.62,5.81,6.02,6.14
2,1.5,0.366116,0.312029,0.304798,0.350100,0.266482,0.315730,0.341681,0.308591,0.296487,...,6.38,6.47,6.31,6.06,5.76,5.30,5.60,5.77,6.04,6.22
3,2.0,0.462916,0.382251,0.360791,0.428252,0.308943,0.366517,0.386763,0.335334,0.316017,...,6.52,6.63,6.45,6.19,5.83,5.33,5.61,5.78,6.05,6.27
4,2.5,0.598794,0.500577,0.460835,0.545139,0.390191,0.457938,0.486965,0.426101,0.369027,...,6.65,6.77,6.57,6.29,5.92,5.41,5.66,5.82,6.11,6.33
5,3.0,0.739882,0.631413,0.576373,0.672529,0.513375,0.581189,0.636467,0.543781,0.436382,...,6.77,6.90,6.70,6.40,6.00,5.50,5.71,5.88,6.18,6.39
6,3.5,0.894528,0.768256,0.699302,0.808232,0.659106,0.728903,0.814293,0.682400,0.519900,...,6.86,7.02,6.81,6.49,6.07,5.59,5.78,5.95,6.25,6.45
7,4.0,1.052811,0.901475,0.820499,0.944529,0.811569,0.888076,1.000516,0.831958,0.616091,...,6.92,7.10,6.89,6.57,6.15,5.67,5.84,6.00,6.31,6.51
8,4.5,1.183655,1.016589,0.929010,1.066441,0.963043,1.042951,1.179970,0.975599,0.718551,...,6.98,7.16,6.95,6.63,6.24,5.74,5.89,6.05,6.36,6.56
9,5.0,1.285697,1.115479,1.025819,1.173339,1.111111,1.189293,1.348912,1.108783,0.824075,...,7.04,7.21,7.01,6.68,6.31,5.81,5.94,6.10,6.40,6.60


In [53]:
raw_liab_ibt_dict_tot_1={}
tr = Raw_FTSE_YC_Tot_1['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot_1[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot_1.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot_1[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot_1[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot_1[col])

952

In [54]:
int_df_tot_1 = pd.DataFrame(raw_liab_ibt_dict_tot_1)
int_df_tot_1 = int_df_tot_1.drop(['Date'], axis=1)
int_df_tot_1 = int_df_tot_1.iloc[:, ::-1]
int_df_tot_1[col]

0      5.98
1      5.98
2      5.98
3      5.98
4      5.98
       ... 
947    7.34
948    7.34
949    7.34
950    7.34
951    7.34
Name: 1995-09-30 00:00:00, Length: 952, dtype: float64

In [55]:
liab_plan_dict_totdf_1={}
for col in int_df_tot_1.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot_1[col][j]/100)**DF[j]))
    liab_plan_dict_totdf_1[col] = temp_pv
liab_plan_dict_totdf_1

{Timestamp('1995-09-30 00:00:00'): 19347976249.798626,
 Timestamp('1995-10-31 00:00:00'): 19663591272.85913,
 Timestamp('1995-11-30 00:00:00'): 20119961742.474697,
 Timestamp('1995-12-31 00:00:00'): 20598895443.988457,
 Timestamp('1996-01-31 00:00:00'): 20478760496.13397,
 Timestamp('1996-02-29 00:00:00'): 19358043733.79895,
 Timestamp('1996-03-31 00:00:00'): 18890359333.32342,
 Timestamp('1996-04-30 00:00:00'): 18347095753.541405,
 Timestamp('1996-05-31 00:00:00'): 18241893716.370724,
 Timestamp('1996-06-30 00:00:00'): 18476392069.191357,
 Timestamp('1996-07-31 00:00:00'): 18354733995.422653,
 Timestamp('1996-08-31 00:00:00'): 18047494591.211964,
 Timestamp('1996-09-30 00:00:00'): 18485155154.390656,
 Timestamp('1996-10-31 00:00:00'): 19167693964.99834,
 Timestamp('1996-11-30 00:00:00'): 19659955709.945225,
 Timestamp('1996-12-31 00:00:00'): 19063385146.146706,
 Timestamp('1997-01-31 00:00:00'): 18832379726.430473,
 Timestamp('1997-02-28 00:00:00'): 18776800051.33405,
 Timestamp('1997

In [56]:
pv_df_totdf_1= pd.DataFrame(liab_plan_dict_totdf_1, index = ['Present Values']).transpose()
np.negative(pv_df_totdf_1['Present Values'])
pv_df_totdf_1

,Present Values
1995-09-30,1.934798e+10
1995-10-31,1.966359e+10
1995-11-30,2.011996e+10
1995-12-31,2.059890e+10
1996-01-31,2.047876e+10
...,...
2021-05-31,3.365854e+10
2021-06-30,3.479317e+10
2021-07-31,3.552982e+10
2021-08-31,3.524508e+10


In [57]:
Plan_Return_YC_Totdf_1 = np.zeros(len(liab_plan_dict_totdf_1)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf_1)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf_1[i] = ((pv_df_totdf_1['Present Values'][i+1])/pv_df_totdf_1['Present Values'][i])-1
Plan_Returns_YC_Totdf_1 = pd.DataFrame(Plan_Return_YC_Totdf_1)
Plan_Returns_YC_Totdf_1

,0
0,0.016313
1,0.023209
2,0.023804
3,-0.005832
4,-0.054726
...,...
307,0.002867
308,0.033710
309,0.021172
310,-0.008014


In [58]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010_1.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf_1.to_excel(writer, sheet_name='returns')
pv_df_totdf_1.to_excel(writer, sheet_name='pv')
writer.save()

In [59]:
Plan_PV_DR_ERF = np.zeros(len(df_t))
for j in range (len(df_t)):
    for i in range (j,len(Time_CF_Tot)):
        Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
Plan_PV_DR_ERF

array([3.27816639e+10, 3.27704998e+10, 3.27593075e+10, 3.27480869e+10,
       3.27303119e+10, 3.27124921e+10, 3.26946272e+10, 3.26767173e+10,
       3.26587623e+10, 3.26407619e+10, 3.26227161e+10, 3.26046248e+10,
       3.25864878e+10, 3.25683051e+10, 3.25500765e+10, 3.25318019e+10,
       3.25087401e+10, 3.24856202e+10, 3.24624419e+10, 3.24392051e+10,
       3.24159097e+10, 3.23925556e+10, 3.23691425e+10, 3.23456704e+10,
       3.23221390e+10, 3.22985482e+10, 3.22748980e+10, 3.22511881e+10,
       3.22223506e+10, 3.21934405e+10, 3.21644573e+10, 3.21354011e+10,
       3.21062716e+10, 3.20770685e+10, 3.20477918e+10, 3.20184412e+10,
       3.19890165e+10, 3.19595177e+10, 3.19299444e+10, 3.19002964e+10,
       3.18655757e+10, 3.18307673e+10, 3.17958711e+10, 3.17608868e+10,
       3.17258143e+10, 3.16906533e+10, 3.16554036e+10, 3.16200650e+10,
       3.15846372e+10, 3.15491200e+10, 3.15135132e+10, 3.14778166e+10,
       3.14370830e+10, 3.13962467e+10, 3.13553073e+10, 3.13142647e+10,
      

In [60]:
Asset_Return = .045
MV_Assets = 13203547000
type(Asset_Return)

float

In [61]:
Plan_Asset_MV = np.zeros(len(df_t))
for i in range(len(df_t)):
        if (i == 0):
            #for i in range (0,len(df_t)):
                Plan_Asset_MV[i] = MV_Assets
        else:
            #for i in range (1,len(df_t)):
                Plan_Asset_MV[i] = (Plan_Asset_MV[i-1]*(1+Asset_Return)**DF[0].tolist())-CF[i-1]
Plan_Asset_MV

array([ 1.32035470e+10,  1.31581944e+10,  1.31126751e+10,  1.30669886e+10,
        1.30146080e+10,  1.29620349e+10,  1.29092686e+10,  1.28563084e+10,
        1.28031536e+10,  1.27498035e+10,  1.26962573e+10,  1.26425143e+10,
        1.25885739e+10,  1.25344352e+10,  1.24800976e+10,  1.24255603e+10,
        1.23660815e+10,  1.23063841e+10,  1.22464674e+10,  1.21863305e+10,
        1.21259726e+10,  1.20653929e+10,  1.20045906e+10,  1.19435648e+10,
        1.18823148e+10,  1.18208397e+10,  1.17591387e+10,  1.16972109e+10,
        1.16299879e+10,  1.15625179e+10,  1.14948000e+10,  1.14268332e+10,
        1.13586166e+10,  1.12901493e+10,  1.12214305e+10,  1.11524591e+10,
        1.10832343e+10,  1.10137550e+10,  1.09440205e+10,  1.08740296e+10,
        1.07987836e+10,  1.07232610e+10,  1.06474609e+10,  1.05713822e+10,
        1.04950240e+10,  1.04183851e+10,  1.03414647e+10,  1.02642615e+10,
        1.01867747e+10,  1.01090031e+10,  1.00309457e+10,  9.95260146e+09,
        9.86902246e+09,  

In [62]:
df_Asset_MV = pd.DataFrame(Plan_Asset_MV)
df_Asset_MV

,0
0,1.320355e+10
1,1.315819e+10
2,1.311268e+10
3,1.306699e+10
4,1.301461e+10
...,...
947,-4.396559e+11
948,-4.412716e+11
949,-4.428932e+11
950,-4.445208e+11


In [63]:
FS = Plan_Asset_MV/Plan_PV_DR_ERF
FS = pd.DataFrame(FS)
FS.style.format("{:.2%}")


,0
0,40.28%
1,40.15%
2,40.03%
3,39.90%
4,39.76%
5,39.62%
6,39.48%
7,39.34%
8,39.20%
9,39.06%


In [64]:
def full_solve(x0, **kwargs):
    return np.asscalar(fsolve(fullfillment_solve, x0=x0, **kwargs))


def fullfillment_solve(fullfillment_return):

SyntaxError: unexpected EOF while parsing (<ipython-input-64-d1790ce9d700>, line 5)